## Extracting predictors from BRFSS

In this notebook, we're going to figure out which predictors we want to extract from the BRFSS CSVs. This is going to take several steps:

1. Load consensus_var_desc_dict which contains all the variable names and descriptions from all the codebooks from 1999 to 2017. Also load the master_codebook_all_years dataframe, which will help you figure out for which years which variables were used.
2. Decide which predictors I want to extract from each year's BRFSS CSV.
3. Load each BRFSS CSV into a pandas dataframe, and extract the relevant columns. Store this is a dictionary or list of dataframe.
4. Then, once you have this list or dictionary of dataframes by year, take each dataframe and compute the state-level summary statistics you want for each year; turn this into a time series for each predictor that has an annual sampling frequency.

In [1]:
%%bash
ls ../data/pickles

MI_mortality_medicaid_expansion.pkl
any_exercise_list_of_dfs.pkl
cardiac_mortality_obesity_dm_df_by_state.pkl
codebook_dfs_dict.pkl
consensus_var_desc_dict.pkl
interpol_truncated_MI_mortality_per_state_dict.pkl
master_codebook_all_years.pkl
myocardial_infarction_df_state_mortality_dict.pkl
state_population_by_year_dict.pkl


In [521]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings

from progress_bar import log_progress

warnings.filterwarnings('ignore')

%matplotlib inline

In [4]:
with open("../data/pickles/master_codebook_all_years.pkl", "rb") as f:
    master_codebook_all_years_df = pickle.load(f)

In [6]:
with open("../data/pickles/consensus_var_desc_dict.pkl", "rb") as f:
    consensus_var_desc_dict = pickle.load(f)

What we need to do is find all the variables that mention a particular condition, so that we can figure out which variable names are synonymous with each other.

In [455]:
master_codebook_all_years_df[master_codebook_all_years_df.var_name == 'SEX']

,var_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
72,SEX,11.1. A mammogram is an x -ray of each breast ...,11.1. A mammogram is an x -ray of each breast ...,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.,Indicate sex of respondent.


In [456]:
relevant_variables = {}
relevant_variables['high_cholesterol'] = ['TOLDHI','TOLDHI2']
relevant_variables['hypertension'] = ['BPHIGH', 'BPHIGH2', 'BPHIGH3', 'BPHIGH4']
# relevant_variables['aspirin'] = ['CVDASPRN']
relevant_variables['exercise'] = ['EXERANY', 'EXERANY2']
relevant_variables['general_health'] = ['GENHLTH']
relevant_variables['mental_health'] = ['MENTHLTH']
relevant_variables['coverage'] = ['HLTHPLAN', 'HLTHPLN1']
relevant_variables['income'] = ['INCOME2']
relevant_variables['smoker'] = ['SMOKER2', 'SMOKER3']
relevant_variables['med_cost'] = ['MEDCOST']
relevant_variables['gender'] = ['SEX']
relevant_variables['weight'] = ['FINALWT', 'LLCPWT']

In [457]:
# This cell goes through all the key and values, and if the condition is mentioned in the description/question,
# it prints the variable name and the description/question.

condition = 'education'

for key, val in consensus_var_desc_dict.items():
    if val:
        if condition in val:
            print(key)
            print(val)
            print("\n")
        if condition.capitalize() in val:
            print(key)
            print(val)
            print("\n")

ARTHEDU
Have you EVER taken an educational course or class to teach you how to manage problems related to your arthritis or joint symptoms?


_EDUCAG
Level of education completed


MISPHLPF
People are generally caring and sympathetic to people with mental illness. Do you agree slightly or strongly or disagree slightly or strongly? (If asked for the purpose of Q9 or Q10, say: "answers to these questions will be used by health planners to help understand public attitudes about mental illness and its treatment and to help guide health education progams.")


_LLCPM03
Combined Land -Line and Cell -Phone Third Margin (Education)


_IMPEDUC
Imputed Education Level (This value is the reported education level or an imputed education level, if the respondent refused to give an education level. The value of the imputed education level will be computed from the sample if the respondent refused to give an education level.)


_IMPEDUC
Imputed Education Level (This value is the reported education lev

Now, let's write a function that will take a list of columns that we want to extract, check it against the columns of a dataframe, and then it returns a dataframe with the columns that this dataframe has that we want to extract. We have to write this function because the dataframes for all of the different years have different variable names.

In [458]:
columns_to_extract = ['STATE']
for key, val in relevant_variables.items():
    columns_to_extract.extend(val)

In [460]:
def find_cols_this_df_has(list_of_columns, df):
    good_cols = []
    for col in df.columns:
        temp_col = col.replace("x.", "")
        for col_to_extract in list_of_columns:
            if col_to_extract.lower() == temp_col.lower():
                good_cols.append(col)
    return good_cols

In [461]:
columns_to_extract

['STATE',
 'TOLDHI',
 'TOLDHI2',
 'BPHIGH',
 'BPHIGH2',
 'BPHIGH3',
 'BPHIGH4',
 'EXERANY',
 'EXERANY2',
 'GENHLTH',
 'MENTHLTH',
 'HLTHPLAN',
 'HLTHPLN1',
 'INCOME2',
 'SMOKER2',
 'SMOKER3',
 'MEDCOST',
 'SEX',
 'FINALWT',
 'LLCPWT']

Now, let's iterate through each of the BRFSS CSVs and load them into a dataframe, and for each dataframe we'll get all the columns that we want to extract that this dataframe contains.

In [462]:
years = list(range(1999, 2018))

In [463]:
dict_of_relevant_dfs = {}

for year in log_progress(years):
    brfss_df = pd.read_csv(f"../data/brfss/csv/brfss{year}.csv", encoding="cp1252")
    cols = find_cols_this_df_has(columns_to_extract, brfss_df)
    print(year)
    print(cols)
    temp_df = brfss_df[cols].copy()
    
    temp_df['year'] = year
    dict_of_relevant_dfs[year] = temp_df

1999
['x.state', 'genhlth', 'menthlth', 'hlthplan', 'medcost', 'bphigh', 'toldhi', 'income2', 'sex', 'exerany', 'x.smoker2', 'x.finalwt']
2000
['x.state', 'genhlth', 'menthlth', 'hlthplan', 'medcost', 'exerany', 'income2', 'sex', 'bphigh', 'toldhi', 'x.smoker2', 'x.finalwt']
2001
['x.state', 'genhlth', 'menthlth', 'hlthplan', 'exerany2', 'bphigh2', 'toldhi2', 'income2', 'sex', 'medcost', 'x.finalwt', 'x.smoker2']
2002
['x.state', 'genhlth', 'hlthplan', 'exerany2', 'income2', 'sex', 'bphigh3', 'toldhi2', 'menthlth', 'x.finalwt', 'x.smoker2']
2003
['x.state', 'genhlth', 'menthlth', 'hlthplan', 'medcost', 'exerany2', 'bphigh3', 'toldhi2', 'income2', 'sex', 'x.finalwt', 'x.smoker2']
2004
['x.state', 'genhlth', 'menthlth', 'hlthplan', 'medcost', 'exerany2', 'income2', 'sex', 'bphigh3', 'toldhi2', 'x.finalwt', 'x.smoker2']
2005
['x.state', 'genhlth', 'menthlth', 'hlthplan', 'medcost', 'exerany2', 'bphigh4', 'toldhi2', 'income2', 'sex', 'x.finalwt', 'x.smoker3']
2006
['x.state', 'genhlth', 'm

In [464]:
with open("../data/pickles/dict_of_relevant_dfs_raw.pkl", "wb") as f:
    pickle.dump(dict_of_relevant_dfs, f)

Now, let's clean up each of the dataframes. Unfortunately, the easiest way to do this on a relatively small set is to do it manually.

In [359]:
with open("../data/pickles/dict_of_relevant_dfs_raw.pkl", "rb") as f:
    dict_of_relevant_dfs = pickle.load(f)

Let's remove the rows where x.state is 66 (Guam) or 72 (Puerto Rico).

In [465]:
for year in years:
    temp_df = dict_of_relevant_dfs[year].copy()
    temp_df = temp_df[(temp_df['x.state'] != 66) & (temp_df['x.state'] != 72) & (temp_df['x.state'] != 78)]
    dict_of_relevant_dfs[year] = temp_df

In [466]:
for year, df in dict_of_relevant_dfs.items():
    
    rename_dict = {}

    for col in df.columns:
        for key, value in relevant_variables.items():
            if col.replace("x.", "").upper() in value:
                rename_dict[col] = key
                
    dict_of_relevant_dfs[year] = df.rename(columns=rename_dict)

Now that we have a dictionary of the dataframes with the predictors that we're interested in, we now have to translate the numbers into the actual responses. We can than calculate the mean per state per year for each of our predictors.

In [467]:
dict_of_relevant_dfs[2017].head()

,x.state,general_health,mental_health,coverage,med_cost,hypertension,high_cholesterol,gender,income,exercise,weight,smoker,year
0,1,2.0,88.0,1.0,2.0,1.0,1.0,2,6.0,1.0,79.425947,4,2017
1,1,2.0,88.0,1.0,2.0,1.0,2.0,1,8.0,1.0,89.694580,4,2017
2,1,3.0,88.0,1.0,2.0,3.0,1.0,1,99.0,2.0,440.121376,4,2017
3,1,4.0,88.0,1.0,2.0,1.0,1.0,2,1.0,NaN,194.867164,4,2017
4,1,4.0,88.0,1.0,2.0,3.0,2.0,2,2.0,2.0,169.087888,3,2017


In [468]:
with open("../data/fips_states.txt", "r") as f:
    lines = f.readlines()

lines = [line.strip() for line in lines]

fips_state = [(int(line.split("\t")[0]), line.split("\t")[1]) for line in lines]

fips_state_dict = dict(fips_state)

In [469]:
var_name_response_num_to_str_dict = {}

var_name_response_num_to_str_dict['general_health'] = {1: 'excellent', 2: 'very good', 3: 'good',
                                                       4: 'fair', 5: 'poor', 7: "don't know", 9: 'refused'}
var_name_response_num_to_str_dict['mental_health'] = {77: "don't know", 88: 0, 99: "refused"}
var_name_response_num_to_str_dict['coverage'] = {1: 'yes', 2: 'no', 7: "don't know", 9: "refused"}
var_name_response_num_to_str_dict['med_cost'] = {1: 'yes', 2: 'no', 7: "don't know", 9: "refused"}
var_name_response_num_to_str_dict['hypertension'] = {1: 'yes', 2: 'yes, only during pregnancy', 3:'no', 4:'borderline', 7: "don't know", 9: "refused"}
var_name_response_num_to_str_dict['high_cholesterol'] = {1: 'yes', 2: 'no', 7: "don't know", 9: "refused"}
var_name_response_num_to_str_dict['gender'] = {1: 'male', 2: 'female', 9: 'refused'}
var_name_response_num_to_str_dict['income'] = {1: 5, 2: 12.5, 3: 17.5, 4: 22.5, 
                                               5: 30, 6: 42.5, 7: 62.5, 8: 100, 77: "don't know", 99: 'refused'}
# var_name_response_num_to_str_dict['income'] = {1: '<10K', 2: '10K-15K', 3: '15K-20K', 4: '20K-25K', 
#                                                5: '25K-35K', 6: '35K-50K', 7: '50K-75K', 8: '>75', 77: "don't know", 99: 'refused'}
var_name_response_num_to_str_dict['exercise'] = {1: 'yes', 2: 'no', 7: "don't know", 9: "refused"}
# var_name_response_num_to_str_dict['aspirin'] = {1: 'yes', 2: 'no', 7: "don't know", 9: "refused"}

var_name_response_num_to_str_dict['smoker'] = {1: 'current, smoke every day', 2: 'current, smoke some days', 3: 'former smoker', 4: 'never smoked', 9: 'refused'}
var_name_response_num_to_str_dict['x.state'] = fips_state_dict

In [470]:
dict_of_relevant_dfs[1999].head()

,x.state,general_health,mental_health,coverage,med_cost,hypertension,high_cholesterol,income,gender,exercise,smoker,weight,year
0,1,1,88,1,2,2.0,2.0,7,2,NaN,3,1419.911405,1999
1,1,2,10,2,1,2.0,2.0,1,2,NaN,4,1839.091587,1999
2,1,2,88,1,2,2.0,2.0,5,2,NaN,4,1024.247333,1999
3,1,2,88,1,2,1.0,2.0,8,1,NaN,4,2381.874267,1999
4,1,2,2,2,2,2.0,2.0,7,1,NaN,4,2381.874267,1999


In [471]:
dict_of_dfs_per_year_with_str_responses = {}

for year in years:
    temp_df = dict_of_relevant_dfs[year].copy()
    for col in temp_df.columns:
        if col in var_name_response_num_to_str_dict:
            temp_df[col] = temp_df[col].replace(to_replace=var_name_response_num_to_str_dict[col])
    dict_of_dfs_per_year_with_str_responses[year] = temp_df

In [472]:
dict_of_dfs_per_year_with_str_responses[2015].head()

,x.state,general_health,mental_health,coverage,med_cost,hypertension,high_cholesterol,gender,income,exercise,weight,smoker,year
0,Alabama,poor,18,yes,no,yes,yes,female,17.5,no,341.384853,former smoker,2015
1,Alabama,good,0,no,yes,no,no,female,5,yes,108.060903,"current, smoke every day",2015
2,Alabama,fair,0,yes,no,no,yes,female,refused,NaN,255.264797,refused,2015
3,Alabama,poor,30,yes,yes,yes,yes,female,100,no,341.384853,never smoked,2015
4,Alabama,poor,0,yes,no,no,no,female,don't know,no,258.682223,never smoked,2015


Now, we need to figure out which response we're interested in for each predictor. E.g., for `high_cholesterol`, we want the percent of people who responded 'yes' in each state, whereas for `coverage` we're interested in the percent of people who respond 'no'. The other thing that we have to do is to weight each response appropriately according to each record's 'weight_to_multiply' (either FINALWT or LLCPWT depending on what year the survey was taken.

In [473]:
temp = dict_of_dfs_per_year_with_str_responses[2015]

In [474]:
temp.head()

,x.state,general_health,mental_health,coverage,med_cost,hypertension,high_cholesterol,gender,income,exercise,weight,smoker,year
0,Alabama,poor,18,yes,no,yes,yes,female,17.5,no,341.384853,former smoker,2015
1,Alabama,good,0,no,yes,no,no,female,5,yes,108.060903,"current, smoke every day",2015
2,Alabama,fair,0,yes,no,no,yes,female,refused,NaN,255.264797,refused,2015
3,Alabama,poor,30,yes,yes,yes,yes,female,100,no,341.384853,never smoked,2015
4,Alabama,poor,0,yes,no,no,no,female,don't know,no,258.682223,never smoked,2015


We will calculate the target value for each variable as follows:
- general_health: binarize so that if they say their general health is fair or poor, they have 'poor health'; we calculate the percent of people with poor health.
- mental_health: this is the weighted mean of the number of days in a month that people experienced 'poor mental health', e.g. anxiety, stress, depression.
- med_cost: the question is in the past year, was there a time you had to see a doctor but couldn't because of cost? We calculate the percent of people who answered yes.
- coverage: the question is, do you have any type of health coverage (private insurance, medicaid, etc.)? We calculate the percept of people who say no.
- hypertension: the question is, has a medical professional ever told you that you have high blood pressure? We calculate the percent of people who say yes.
- high_cholesterol: the question is, has a medical professional ever told you that you have high cholesterol? We calculate the percent of people who say yes.
- gender: we calculate the percent of people who are male.
- income: we take the weighted average of the salary bins that people identified as their salaries. This is probably underestimating the salaries because the denominator includes the 'weight' for everybody in the state, even people who refused or don't know their salary.
- exercise: the question is, have you done any exercise in the past 30 days? We calculate the percent of people who say no.
- aspirin: this was going to calculate the percent of people who take aspirin daily or every other day, but there was too little data; most states didn't ask this question.
- smoker: this calculates the percent of people who say they are current smokers (either smoke every day or some days).

In [480]:
def calculate_target_value(var_name, df):
    total_weight_for_group = sum(df.weight)
    if var_name not in df.columns:
        return None
    
    if var_name == 'general_health':
        r = sum(df[(df.general_health == 'fair') | (df.general_health == 'poor')]['weight'])/total_weight_for_group
    elif var_name == 'mental_health':
        # For all the people who responded with a number when asked how many days of poor mental health they experienced, we multiply it by the weight, drop the NaNs, sum it, and divide by the total weight.
        r = sum((df.mental_health.replace({"refused": np.nan, "don't know": np.nan}) * df.weight).dropna())/total_weight_for_group
    elif var_name == 'med_cost':
        r = sum(df[df.med_cost == 'yes']['weight'])/total_weight_for_group
    elif var_name == 'coverage':
        r = sum(df[df.coverage == 'no']['weight'])/total_weight_for_group
    elif var_name == 'hypertension':
        r = sum(df[df.hypertension == 'yes']['weight'])/total_weight_for_group
    elif var_name == 'high_cholesterol':
        r = sum(df[df.high_cholesterol == 'yes']['weight'])/total_weight_for_group
    elif var_name == 'gender':
        r = sum(df[df.gender == 'male']['weight'])/total_weight_for_group
    elif var_name == 'income':
        r = sum((df.income.replace({'refused': np.nan, "don't know": np.nan}) * df.weight).dropna())/total_weight_for_group
    elif var_name == 'exercise':
        r = sum(df[df.exercise == 'no']['weight'])/total_weight_for_group
    elif var_name == 'aspirin':
        r = sum(df[df.aspirin == 'yes']['weight'])/total_weight_for_group
    elif var_name == 'smoker':
        r = sum(df[(df.smoker == 'current, smoke every day') | (df.smoker == 'current, smoke some days')]['weight'])/total_weight_for_group
    
    if r == 0:
#         print('Calculated value for this variable was 0. This likely means that this predictor contained only NaNs.')
        return None
    else:
        return r

In [481]:
calculate_target_value('general_health', alabama)

0.22102490818659773

In [483]:
calculate_target_value('med_cost', alabama)

0.16443652309130397

In [484]:
calculate_target_value('coverage', alabama)

0.13032736039086454

Now, we need to iterate through the dataframe for each year, groupby state, and then calculate the target value for each state for that year, and store it.

In [485]:
list_of_predictors_to_calculate = list(relevant_variables.keys())
list_of_predictors_to_calculate.remove('weight')

In [486]:
print(list_of_predictors_to_calculate)

['high_cholesterol', 'hypertension', 'exercise', 'general_health', 'mental_health', 'coverage', 'income', 'smoker', 'med_cost', 'gender']


In [487]:
tuple_to_df_list = []

for year in log_progress(years):
    year_df = dict_of_dfs_per_year_with_str_responses[year].copy()
    groupby_state = year_df.groupby(by='x.state')
    
    for state in groupby_state.groups:
        state_df = groupby_state.get_group(state)
        for var in list_of_predictors_to_calculate:
            target_value = calculate_target_value(var, state_df)
            tuple_to_df_list.append( (state, year, var, target_value) )

In [488]:
state_year_var_name_val_df = pd.DataFrame(tuple_to_df_list, columns=['state', 'year', 'var_name', 'value'])

In [490]:
year_var_by_state_gb = state_year_var_name_val_df.groupby(['state', 'var_name'])[['year', 'value']]

In [491]:
dict_of_state_var_dfs = {}

for group in year_var_by_state_gb.groups:
    state = group[0]
    var_name = group[1]
    
    if state not in dict_of_state_var_dfs:
        dict_of_state_var_dfs[state] = year_var_by_state_gb.get_group(group).rename(columns={'value':var_name})
#         print(dict_of_state_var_dfs[state])
    else:
        dict_of_state_var_dfs[state] = pd.merge(dict_of_state_var_dfs[state], year_var_by_state_gb.get_group(group).rename(columns={'value':var_name}), on='year')
#         print(dict_of_state_var_dfs[state])

In [535]:
dict_of_interpol_covariate_state_dfs = {}

for state, df in dict_of_state_var_dfs.items():
    temp = df.copy()
    temp = temp.interpolate(method='linear')
    temp = temp.fillna(temp.mean())
    
    temp.year = pd.to_datetime(temp.year, format="%Y")
    temp = temp.set_index('year').resample('M').ffill(limit=1).interpolate('linear')
    temp.index = temp.index - pd.offsets.MonthBegin(0) - pd.DateOffset(months=1)
    
    dict_of_interpol_covariate_state_dfs[state] = temp

In [537]:
dict_of_interpol_covariate_state_dfs['California'].head()

,high_cholesterol,hypertension,exercise,general_health,mental_health,coverage,income,smoker,med_cost,gender
year,,,,,,,,,,
1999-01-01,0.210998,0.228108,0.212816,0.158018,3.234794,0.193080,40.585070,0.187049,0.111224,0.495518
1999-02-01,0.211884,0.228286,0.217142,0.158739,3.233021,0.192317,40.847403,0.185803,0.112605,0.495578
1999-03-01,0.212769,0.228463,0.221469,0.159460,3.231247,0.191554,41.109736,0.184557,0.113986,0.495638
1999-04-01,0.213654,0.228641,0.225795,0.160181,3.229474,0.190790,41.372069,0.183310,0.115367,0.495697
1999-05-01,0.214540,0.228819,0.230121,0.160902,3.227701,0.190027,41.634402,0.182064,0.116748,0.495757


In [538]:
with open("../data/pickles/dict_of_interpol_covariate_state_dfs.pkl", "wb") as f:
    pickle.dump(dict_of_interpol_covariate_state_dfs, f)